In [18]:
!pip install boto3
!pip install pandas
!pip install joblib
!pip install scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import boto3
import os
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sagemaker.sklearn.estimator import SKLearn

In [14]:
# # SageMaker environment variables
# input_dir = os.environ['SM_CHANNEL_TRAIN']       # where training data is downloaded
# model_dir = os.environ['SM_MODEL_DIR']           # where model must be saved

# Combining the CSVs

In [21]:
# === CONFIGURATION ===
csv_dir = 'cleaned_data/'  # Replace with your folder path
output_file = 'combinedHistoricalData.csv'   # Output filename

# === COMBINE CSV FILES ===
all_dfs = []
for filename in os.listdir(csv_dir):
    if filename.endswith('.csv'):
        filepath = os.path.join(csv_dir, filename)
        df = pd.read_csv(filepath)
        df['source_file'] = filename  # Optional: track file origin
        all_dfs.append(df)

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df.to_csv(output_file, index=False)

print(f"Combined {len(all_dfs)} files into {output_file}")

Combined 48 files into combinedHistoricalData.csv


In [ ]:
# Load data
df = pd.read_csv(f"{input_dir}/train.csv")
X = df.drop("label", axis=1)
y = df["label"]

In [ ]:
# Train model
model = RandomForestClassifier()
model.fit(X, y)

In [ ]:
# Save model
joblib.dump(model, f"{model_dir}/model.joblib")

In [ ]:
sklearn = SKLearn(entry_point='train.py',
                  role='SageMakerRole',
                  instance_type='ml.m5.large',
                  framework_version='1.0-1',
                  py_version='py3')

sklearn.fit({'train': 's3://csu-summer-camp-electricity-2025\data-csv'})

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
/var/folders/3b/9d3900dj0b53yzgc017pbr480000gq/T/ipykernel_98878/4197486081.py:7: SyntaxWarning: invalid escape sequence '\d'
  sklearn.fit({'train': 's3://csu-summer-camp-electricity-2025\data-csv'})
/var/folders/3b/9d3900dj0b53yzgc017pbr480000gq/T/ipykernel_98878/4197486081.py:7: SyntaxWarning: invalid escape sequence '\d'
  sklearn.fit({'train': 's3://csu-summer-camp-electricity-2025\data-csv'})


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│   4 │   │   │   │     framework_version='1.0-1',                                                 │
│   5 │   │   │   │     py_version='py3')                                                          │
│   6                                                                                              │
│ ❱ 7 sklearn.fit({'train': 's3://csu-summer-camp-electricity-2025\data-csv'})                     │
│   8                                                                                              │
│                                                                                                  │
│ /Users/carlos_1/Documents/GitHub/aws-slo-camp-repos/aws-ai-cpslo-energy-efficiency/.venv/lib/pyt │
│ hon3.13/site-packages/sagemaker/telemetry/telemetry_logging.py:168 in wrapper                    │
│                                                                                                  │
│   165 │   │   │   │   │   caught_ex = e                                                          │
│   166 │   │   │   │   finally:                                                                   │
│   167 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 168 │   │   │   │   │   │   raise caught_ex                                                    │
│   169 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   170 │   │   │   else:                                                                          │
│   171 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /Users/carlos_1/Documents/GitHub/aws-slo-camp-repos/aws-ai-cpslo-energy-efficiency/.venv/lib/pyt │
│ hon3.13/site-packages/sagemaker/telemetry/telemetry_logging.py:139 in wrapper                    │
│                                                                                                  │
│   136 │   │   │   │   start_timer = perf_counter()                                               │
│   137 │   │   │   │   try:                                                                       │
│   138 │   │   │   │   │   # Call the original function                                           │
│ ❱ 139 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   140 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   141 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   142 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /Users/carlos_1/Documents/GitHub/aws-slo-camp-repos/aws-ai-cpslo-energy-efficiency/.venv/lib/pyt │
│ hon3.13/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper                      │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                   

In [ ]:
predictor = sklearn.deploy(instance_type='ml.m5.large', initial_instance_count=1)
predictor.predict([[1.0, 2.0, 3.0]])